#### Предсказываем цену автомобиля

В папке лежит датасет Car details v3, в котором содержатся сведения об автомобилях, их характеристики и цены. Обучите модель линейной регрессии, которая будет предсказывать цену автомобиля по его характеристикам.

Придется хорошенько поработать с характеристиками: это *творческая* часть задания. Во-первых, надо привести их в машиночитаемый вид, а во-вторых, можно посмотреть, как они коррелируют друг с другом и не нужно ли кого-то из них дропнуть или наоборот.

*Примечание*: без работы над фичами за все дз - **0 баллов**.

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data = pd.read_csv('/content/gdrive/My Drive/data_stud/machlearn/Car details v3.csv')
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [ ]:
print (data['fuel'].unique())
print (data['seller_type'].unique())
print (data['owner'].unique())
print (data['transmission'].unique())

['Diesel' 'Petrol' 'LPG' 'CNG']
['Individual' 'Dealer' 'Trustmark Dealer']
['First Owner' 'Second Owner' 'Third Owner' 'Fourth & Above Owner'
 'Test Drive Car']
['Manual' 'Automatic']


In [ ]:
#fuel
data.loc[data["fuel"] == "Diesel", "fuel"] = 0
data.loc[data["fuel"] == "Petrol", "fuel"] = 1
data.loc[data["fuel"] == "LPG", "fuel"] = 2
data.loc[data["fuel"] == "CNG", "fuel"] = 3

#seller_type
data.loc[data["seller_type"] == "Individual", "seller_type"] = 0
data.loc[data["seller_type"] == "Dealer", "seller_type"] = 1
data.loc[data["seller_type"] == "Trustmark Dealer", "seller_type"] = 2

#owner
data.loc[data["owner"] == "First Owner", "owner"] = 0
data.loc[data["owner"] == "Second Owner", "owner"] = 1
data.loc[data["owner"] == "Third Owner", "owner"] = 2
data.loc[data["owner"] == "Fourth & Above Owner", "owner"] = 3
data.loc[data["owner"] == "Test Drive Car", "owner"] = 4

#transmission
data.loc[data["transmission"] == "Manual", "transmission"] = 0
data.loc[data["transmission"] == "Automatic", "transmission"] = 1

#mileage
data['mileage'] = data['mileage'].str.replace(r'[^0-9&.]', '', regex=True)

#engine
data['engine'] = data['engine'].str.replace(r'[^0-9&.]', '', regex=True)

#max_power
data['max_power'] = data['max_power'].str.replace(r'[^0-9&.]', '', regex=True)

data.head ()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,0,0,0,0,23.4,1248,74,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,0,0,0,1,21.14,1498,103.52,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,1,0,0,2,17.7,1497,78,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,0,0,0,0,23.0,1396,90,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,1,0,0,0,16.1,1298,88.2,"11.5@ 4,500(kgm@ rpm)",5.0


In [ ]:
#torque
data['torque'] = data['torque'].str.replace(r' at', '@', regex=True)
mask = data['torque'].str.contains(r'kgm|KGM', na=True) #список ячеек, которые нужно конвертировать в другую величину...
print (mask)
df = data

df['torque'] = df['torque'].str.replace(r'[^0-9&@&.]', '', regex=True)
df['torque'] = df['torque'].str.replace(r'@$', '', regex=True)
df[['Nm', 'rpm']] = df['torque'].str.split('@', expand=True) #разделили torque на две разные величины
df = df.drop(['torque'], axis=1)



i = 0
num = 9.806
nm = df['Nm'].tolist()
while (i < len(mask)):
  if (mask[i] == True):
    nm[i] = float(nm[i])*num #kgm в Nm
  i+=1

df['Nm'] = nm



df.head ()
#print (mask)

0       False
1       False
2        True
3        True
4        True
        ...  
8123    False
8124     True
8125    False
8126    False
8127    False
Name: torque, Length: 8128, dtype: bool


,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats,Nm,rpm
0,Maruti Swift Dzire VDI,2014,450000,145500,0,0,0,0,23.4,1248,74,5.0,190,2000
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,0,0,0,1,21.14,1498,103.52,5.0,250,15002500
2,Honda City 2017-2020 EXi,2006,158000,140000,1,0,0,2,17.7,1497,78,5.0,124.5362,2700
3,Hyundai i20 Sportz Diesel,2010,225000,127000,0,0,0,0,23.0,1396,90,5.0,219.6544,17502750
4,Maruti Swift VXI BSIII,2007,130000,120000,1,0,0,0,16.1,1298,88.2,5.0,112.769,4500


In [ ]:
data = df #сохраняем это нормально

In [ ]:
rpm = df['rpm'].tolist()
i = 0

while (i < len(rpm)):
  if (len(str(rpm[i]))) > 4:
    rpm[i] = rpm[i][:-4]
  i+=1

df['rpm'] = rpm

data = df

df.head ()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats,Nm,rpm
0,Maruti Swift Dzire VDI,2014,450000,145500,0,0,0,0,23.4,1248,74,5.0,190,2000
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,0,0,0,1,21.14,1498,103.52,5.0,250,1500
2,Honda City 2017-2020 EXi,2006,158000,140000,1,0,0,2,17.7,1497,78,5.0,124.5362,2700
3,Hyundai i20 Sportz Diesel,2010,225000,127000,0,0,0,0,23.0,1396,90,5.0,219.6544,1750
4,Maruti Swift VXI BSIII,2007,130000,120000,1,0,0,0,16.1,1298,88.2,5.0,112.769,4500


In [ ]:
df = data
df.dropna()
df = df.drop (['name'], axis=1)
df.head ()

,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats,Nm,rpm
0,2014,450000,145500,0,0,0,0,23.4,1248,74,5.0,190,2000.0
1,2014,370000,120000,0,0,0,1,21.14,1498,103.52,5.0,250,1500.0
2,2006,158000,140000,1,0,0,2,17.7,1497,78,5.0,124.5362,2700.0
3,2010,225000,127000,0,0,0,0,23.0,1396,90,5.0,219.6544,1750.0
4,2007,130000,120000,1,0,0,0,16.1,1298,88.2,5.0,112.769,4500.0


In [ ]:
df = df.dropna ()
df.isna().any()

year             False
selling_price    False
km_driven        False
fuel             False
seller_type      False
transmission     False
owner            False
mileage          False
engine           False
max_power        False
seats            False
Nm               False
rpm              False
dtype: bool

In [ ]:
from sklearn import preprocessing
x = df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df1=pd.DataFrame(x_scaled, columns=df.columns)

df1

,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats,Nm,rpm
0,0.769231,0.042126,0.061640,0.000000,0.0,0.0,0.00,0.557143,0.209396,0.112200,0.25,0.000383,0.399760
1,0.769231,0.034102,0.050837,0.000000,0.0,0.0,0.25,0.503333,0.293289,0.192593,0.25,0.000544,0.299720
2,0.461538,0.012839,0.059310,0.333333,0.0,0.0,0.50,0.421429,0.292953,0.123094,0.25,0.000208,0.539816
3,0.615385,0.019559,0.053803,0.000000,0.0,0.0,0.00,0.547619,0.259060,0.155773,0.25,0.000463,0.349740
4,0.500000,0.010030,0.050837,0.333333,0.0,0.0,0.00,0.383333,0.226174,0.150871,0.25,0.000176,0.899960
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7859,0.730769,0.029087,0.046601,0.333333,0.0,0.0,0.00,0.440476,0.192282,0.136302,0.25,0.000179,0.799920
7860,0.500000,0.010532,0.050414,0.000000,0.0,0.0,0.75,0.400000,0.291611,0.210240,0.25,0.000505,0.379752
7861,0.576923,0.035306,0.050837,0.000000,0.0,0.0,0.00,0.459524,0.209396,0.111928,0.25,0.000383,0.399760
7862,0.730769,0.026078,0.010591,0.000000,0.0,0.0,0.00,0.561190,0.259060,0.101307,0.25,0.000249,0.359744


In [ ]:
from sklearn.model_selection import train_test_split

X = df1[['year', 'km_driven', 'fuel', 'seller_type', 'transmission', 'owner', 'mileage', 'engine', 'max_power', 'seats', 'Nm', 'rpm']]
y = df1['selling_price']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2)

X.shape

(7864, 12)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()

model.fit(Xtrain, ytrain)

LinearRegression()

In [ ]:
pred_train = model.predict(Xtrain)
pred_test = model.predict(Xtest)
print(mean_squared_error(pred_test, ytest) ** 0.5, mean_squared_error(pred_train, ytrain) ** 0.5)

0.041198955372456735 0.04670798003650629
